#### ========================
#### **1. Library import**  - استيراد المكتبات
#### ========================

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import shapiro

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### ========================
#### **2. Download data** - تحميل البيانات
#### ========================

In [ ]:

df = pd.read_csv("/kaggle/input/heart-disease-cleveland-uci/heart_cleveland_upload.csv")
print(df.shape)
df.head()

# **Meanings of abbreviations -** معاني الاختصارات
##### **age** : age in years
##### **sex** : sex (1: male; 0: female)
##### **cp** : chest pain type (0: typical angina 1: atypical angina 2: non-anginal pain 3: asymptomatic)
##### **trestbps** : resting blood pressure
##### **chol** : serum cholestoral in mg/dl
##### **fbs** : fasting blood sugar > 120 mg/dl (1: true; 0: false)
##### **restecg** : resting electrocardiographic results (0: normal 1: having ST-T wave abnormality (T wave 
##### **thalach** : maximum heart rate achieved
##### **exang** : exercise induced angina (1: yes; 0: no)
##### **oldpeak** : ST depression induced by exercise relative to rest
##### **slope** : the slope of the peak exercise ST segment (0: upsloping 1: flat 2: downsloping)
##### **ca** : number of major vessels (0-3) colored by flourosopy
##### **thal** : thal (0: normal; 1: fixed defect; 2: reversable defect)
##### **condition** : disease condition (0: No disease; 1: disease)

#### ========================================================================
#### **3. Initial exploration and Data Cleaning** - استكشاف أولي وتنظيف البيانات
#### ========================================================================

In [ ]:

df.info()
df.describe()

print("========================================================================")
print("========================================================================")
# Number of missing values in each column - عدد القيم المفقودة في كل عمود
df.isnull().sum()



In [ ]:
# Delete rows with missing values - حذف الصفوف اللي فيها قيم مفقودة
# df= df.dropna()

# or compensate it - أو تعويضها
# df['num_col'] = df['num_col'].fillna(df['num_col'].mean())
# df['cat_col'] = df['cat_col'].fillna(df['cat_col'].mode()[0])

# There is no Missing Values in this Datasets

In [ ]:
#  Delete duplicates - حذف المكررات

df=df.drop_duplicates()

# Change data type (if necessary) - تغيير نوع البيانات (لو لزم)
# df['date_col'] = pd.to_datetime(df['date_col'])

# oldpeak type is float64 we will change it to int if it necessary - نوع oldpeak هو float64 وسنقوم بتغييره إلى int إذا لزم الأمر
# df['oldpeak'] = df['int_col'].astype(int)

In [ ]:
# Handling Outliers Using IQR - معالجة القيم الشاذة (Outliers) باستخدام IQR
plt.figure(figsize=(12,6))
plt.boxplot(df, labels=df.columns)
plt.title("Boxplot of ALL Dataset to check the Outliers")
plt.xticks(rotation=45 , fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel("Values")
plt.show()
# There is an outlier on chol column - هناك قيمة شاذة في عمود الكولسترول

Q1 = df['chol'].quantile(0.25)
Q3 = df['chol'].quantile(0.75)
IQR = Q3-Q1
df_clean = df[~((df['chol'] < (Q1 - 1.5 * IQR)) | (df['chol'] > (Q3 + 1.5 * IQR)))]

# Z-score
z_scores = np.abs(stats.zscore(df['chol'].dropna()))
df_z = df[(z_scores < 3)]

# Boxplot after cleaning - الرسم بعد التنظيف
print(df_clean.max())
plt.figure(figsize=(12,6))
plt.boxplot(df_clean, labels=df_clean.columns)
plt.title("Boxplot of ALL Dataset after remove the Outliers")
plt.xticks(rotation=45 , fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel("Values")
plt.show()
  

In [ ]:
# Make the df_clean same as df - جعل df_clean هو نفسه df
df = df_clean

plt.figure(figsize=(12,6))
plt.boxplot(df, labels=df.columns)
plt.title("Boxplot of ALL Dataset to check if df_clean == df (cleaning the Outliers and put the cleaning data into df again)")
plt.xticks(rotation=45 , fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel("Values")
plt.show()

In [ ]:
# Categorical data coding - ترميز البيانات الفئوية
# Label Encoding
# le = LabelEncoder()
# df['cat_col'] = le.fit_transform(df['cat_col'])

# أو One-Hot Encoding
# df = pd.get_dummies(df, columns=['cat_col'])

# There is no categorical data so we will not use this section - لا يوجد بيانات فئوية لذى لن نستخدم هذا القسم 


In [ ]:
# Normalization/Scaling of Numerical Data - تطبيع/مقياس البيانات العددية
# scaler = StanderScaler()
# df[['num_col','num_col2']] = scaler.fit_transform(df[['num_col1','num_col2']])

# Or MinMax - بديل MinMax 
# minmax = MinMaxScaler()
# df[['num_col','num_col2']] = minmax.fit_transform(df[['num_col1','num_col2']])

# The Data Is Already Clean So we will not use this section also - البيانات نظيفة بالفعل اذن لن نستخدم هذا القسم أيضًا

#### ========================
#### **4. توزيع الهدف (Target)**
#### ========================

In [ ]:
# ========================
# 
# ========================
sns.countplot(x="condition", data=df)
plt.title("Distribution of Heart Disease")
plt.show()

#### ========================
#### **5. Patient age analysis** - تحليل أعمار المرضى
#### ========================

In [ ]:

plt.figure(figsize=(8,5))
sns.histplot(df['age'], kde=True , bins=70)
plt.title("Age Distribution")
plt.show()

In [ ]:
df['sex']

#### ========================
#### **6. Gender and status comparison** - مقارنة بين الجنسين والحالة
#### ========================

In [ ]:

# Change gender from 0/1 to Male/Female - تحويل الجنس من 0/1 إلى Male/Female
df['sex'] = df['sex'].map({0: 'Female', 1: 'Male'})

# Countplot of target for each gender -  رسم Countplot للحالة (target) لكل جنس

sns.countplot(x='sex' , hue='condition', data=df , palette={0:"skyblue", 1:"salmon"}) # 0 = غير مصاب (أزرق)، 1 = مصاب (أحمر)
plt.title("Heart Disease By Gender ")
plt.legend(title='Heart Disease', labels=['No Disease','Disease'])
plt.show()

In [ ]:
# Restoring old sex representation (Male = 1 , Female = 0) - استعادة التمثيل القديم (Male = 1، Female = 0) 
df['sex'] = df['sex'].map({'Female':0 , 'Male': 1})
df['sex']

#### ========================
#### **7. Correlation analysis** - تحليل الارتباط
#### ========================

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm',linewidth=0.9, linecolor='black',fmt='.3f')
plt.title('Correlation Heatmap')
plt.show()

#### ========================
#### **8. In-depth statistical analysis** - تحليل إحصائي معمَّق
#### ========================

In [ ]:
# 8.1 Explore and formulate questions - استكشاف وصياغة الأسئلة

# Count categorical and numerical variables - عد المتغيرات الفئوية والعددية
num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()

print(num_cols)
print("The dataset has a",len(num_cols),"of numerical variables")
print("=========================================================================================")
print(cat_cols)
print("The dataset has a",len(cat_cols),"of categorical variables")

In [ ]:
# 8.2 Check missing values and their pattern - فحص القيم المفقودة ونمطها

# Number of missing values - عدد القيم المفقودة
df.isnull().sum()

# Missing percentage per column - نسبة مفقود لكل عمود
(df.isnull().mean()*100).sort_values(ascending=False)

# Show missing mode (optional: you need missingno library) - عرض نمط المفقودات (اختياري: تحتاج مكتبة missingno)
# import missingno as msno
# msno.matrix(df)

In [ ]:
# 8.3 تحليل أحادي المتغير - Univariate Analysis


# Example of a numerical column - مثال لعمود عددي
col = 'age'
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.histplot(df[col].dropna(), kde=True)
plt.title(f'{col} distribution')

plt.subplot(1,2,2)
sns.boxplot(x=df[col])
plt.title(f'{col} boxplot')
plt.show()

# Additional statistics - إحصاءات إضافية
print('skewness', df[col].skew())
print('kurtosis', df[col].kurtosis())

# categorical variable - متغير فئوي
df['sex'] = df['sex'].map({0: 'Female', 1: 'Male'}) # Mapping sex column from numerical to categorical for the visualization
cat = 'sex'
sns.countplot(x=cat, data=df)
plt.show()
print(df[cat].value_counts(normalize=True)*100) # Percentage % - نسب %


In [ ]:
# Restoring old sex representation (Male = 1 , Female = 0) - استعادة التمثيل القديم (Male = 1، Female = 0) 
df['sex'] = df['sex'].map({'Female':0 , 'Male': 1})
df['sex']

In [ ]:
# 8.4 Bivariate Analysis — Numerical vs. Objective - تحليل ثنائي المتغير (Bivariate) — عددية vs هدف

# Numerical relationship to condition: boxplots divided by target, scatter plot with slope line. - علاقة عددية بالحالة: boxplots مقسومين حسب الهدف، scatter مع خط ميل.

# If the goal is 0/1  -  إذا كان الهدف ثنائي 0/1
df['condition'] = df['condition'].map({0: 'No disease', 1: 'disease'}) # Mapping condition column from numerical to categorical for the visualization
sns.boxplot(x='condition', y='age', data=df)  # age vs target
plt.title('Age by Condition ((0: No disease; 1: disease))')
plt.show()

# numerical-numerical relationship -  علاقة عددية-عددية
sns.scatterplot(x='chol',y='age' , hue='condition', data=df)
plt.show()

In [ ]:
# Restoring old disease condition representation (disease = 1 , No disease = 0) - استعادة التمثيل القديم (disease = 1 , No disease = 0) 
df['condition'] = df['condition'].map({'No disease':0 , 'disease': 1})
df['condition']

In [ ]:
# 8.5 Categorical vs Target Analysis - تحليل فئوي مقابل هدف (Categorical vs Target)

# Frequency tables, ratios, and χ² test for tables. - جداول تكرارية، نسب، واختبار χ² للجداول.

# Frequency - ترددات
pd.crosstab(df['sex'], df['condition'], margins=True)

# relatively - نسبيًا
pd.crosstab(df['sex'], df['condition'], normalize='index')


# Chi-square test for the presence of a relationship between sex and condition - اختبار كاي-تربيع لوجود علاقة بين الجنس والحالة
ct = pd.crosstab(df['sex'], df['condition'])
chi2, p, dof, ex = chi2_contingency(ct)
print('chi2=', chi2, 'p-value', p)

بما أن p-value  = 2.5203903761037046e-06 = 0.00000252
p≪0.05
p≪0.05

نرفض الفرضية الصفرية H0 (التي تقول "لا يوجد علاقة بين الجنس والحالة").

نستنتج أنه **يوجد علاقة ذات دلالة إحصائية** بين الجنس والحالة في البيانات.

__________________________________________________________________________________

p-value = 2.5203903761037046e-06 = 0.00000252

p≪0.05

p≪0.05

We reject the null hypothesis H0 (which states "there is no relationship between gender and condition").

We conclude that **there is a statistically significant relationship** between gender and status in our data.

In [ ]:
# 8.6 Comparison of means (t-test / ANOVA) - مقارنة المتوسطات (t-test / ANOVA)

# t-test Example: Is the mean age different between the two cases? - t-test مثال: هل متوسط العمر مختلف بين الحالتين؟
group0 = df[df['condition']==0]['age'].dropna()
group1 = df[df['condition']==1]['age'].dropna()
tstat, pval = ttest_ind(group0, group1, equal_var=False) # welch's t-test
print('t=',tstat, 'p=', pval)


#  ANOVA Example (If more than 2 groups) - ANOVA مثال (لو أكثر من 2 مجموعة)
model = ols('age ~ C(cp)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

#### أولًا: اختبار T-test

t = -4.22: القيمة سالبة بسبب اتجاه الفرق، ولكن الأهم هو مقدارها الذي يعد كبيرًا نسبيًا.

p = 3.27e-05 أي ما يعادل 0.0000327، وهي قيمة أصغر بكثير من 0.05.
هذا يدل على أن الفرق بين المجموعتين التي تمت مقارنتهما ذو دلالة إحصائية قوية، أي أن احتمال أن يكون الفرق عشوائيًا ضعيف جدًا.







#### ثانيًا: اختبار ANOVA لمتغير cp (نوع ألم الصدر)

F = 3.52 مع قيمة احتمالية (p-value) = 0.0155.

بما أن هذه القيمة أقل من 0.05، فهذا يعني أن هناك فروقًا ذات دلالة إحصائية بين متوسطات الأعمار حسب أنواع متغير cp.

مع ذلك، اختبار ANOVA لا يحدد بين أي المجموعات تحديدًا توجد الفروق، بل يكتفي بالإشارة إلى أن المتوسطات ليست جميعها متساوية.







####  ثالثًا: الفحص اللاحق (Post-hoc test)

لمعرفة أي المجموعات تختلف عن بعضها، يمكن استخدام اختبار Tukey HSD، وذلك على النحو التالي:

====================================================================================

**First: T-test**


 t = -4.22: The value is negative because of the direction of the difference, but what is more important is its magnitude, which is relatively large.



 p = 3.27e-05 which is equal to 0.0000327, which is a value much smaller than 0.05.

- This indicates that the difference between the two groups being compared is strongly statistically significant, i.e., the possibility of the difference being random is very small.



**Second: ANOVA test for the cp variable (type of chest pain)**


 F = 3.52 with p-value = 0.0155.

- Since this value is less than 0.05, it means that there are statistically significant differences between the age means by the types of the cp variable.

- However, the ANOVA test does not determine between which groups the differences exist, but rather indicates that not all means are equal.



**Third: Post-hoc test**

To find out which groups differ from each other, the Tukey HSD test can be used, as follows:

In [ ]:
tukey = pairwise_tukeyhsd(endog=df['age'], groups=df['cp'], alpha=0.05)
print(tukey)

### chest pain type (0: typical angina 1: atypical angina 2: non-anginal pain 3: asymptomatic)

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x='cp', y='age', data=df, palette='Set2')
sns.stripplot(x='cp', y='age', data=df, color='black', alpha=0.4)

plt.title("Comparison of ages by type of chest pain (cp)", fontsize=14)
plt.xlabel("Type of chest pain (cp)", fontsize=12)
plt.ylabel("age", fontsize=12)
plt.show()

In [ ]:
# Effect size (Cohen's d) for comparing two groups -  Effect size (Cohen’s d) للمقارنة بين مجموعتين
def cohens_d(x,y):
    nx, ny = len(x), len(y)
    dof = nx + ny - 2
    pooled_std = np.sqrt(((nx-1)*x.var(ddof=1) + (ny-1)*y.var(ddof=1))/dof)
    return (x.mean() - y.mean()/pooled_std)

print('Cohen d:', cohens_d(group1, group0))

In [ ]:
# 8.7 Correlation Matrix and its interpretation - مصفوفة الارتباط وتفسيرها (Correlation Matrix)

# Use Pearson/Spearman, and draw a heatmap. - استخدام Pearson/ Spearman، ورسم heatmap.

# correlation (Pearson) for numerical columns(all is numerical in this case) - correlation (Pearson) للأعمدة العددية
corr = df.corr(method='pearson')
plt.figure(figsize=(12,8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Pearson Correlation')
plt.show()


# If there is a strong nonlinear relationship, use Spearman. - إذا توجد علاقة غير خطية قوية استخدم Spearman
spearman_corr = df.corr(method='spearman')

In [ ]:
# 8.8 Hypothesis testing and normal distribution condition - فحص الفرضية وشرط التوزيع الطبيعي

# Normal tests: Shapiro-Wilk or Kolmogorov-Smirnov (for small/large groups). - اختبارات نورمالية: Shapiro-Wilk أو Kolmogorov-Smirnov (لمجموعات صغيرة/كبيرة).
stat, p = shapiro(df['age'].dropna())
print('Shapiro age p-value: ', p)

In [ ]:
# If the data isn't natural and we want to compare averages: Use non-standard tests like Mann-Whitney U instead of t-test. - إن لم تكن البيانات طبيعية وكنا نريد مقارنة المتوسطات: استخدم اختبارات لا معيارية مثل Mann-Whitney U بدلاً من t-test.
from scipy.stats import mannwhitneyu

u_stat, p = mannwhitneyu(group0 , group1)
print('Mann-Whitney p:', p)

In [ ]:
# 8.9 Transformations to improve distribution - تحويلات لتحسين التوزيع (Transformations)

# Log example of positive distribution - مثال عمل لوغ على التوزيع الإيجابي
df['chol_log'] = np.log1p(df['chol'])

# Box-Cox (requires values >0) - Box-Cox (يتطلب قيم >0)
from scipy.stats import boxcox
df['col_boxcox'], lam = boxcox(df['chol'].dropna()+1) #if zero add + 1  |  +1 لو صفر
print("Lambda Selected:", lam)

لو كانت λ = 1 → البيانات أصلًا طبيعية تقريبًا (لا حاجة إلى تحويل كبير).

لو كانت λ = 0 → التحويل يشبه التحويل اللوغاريتمي (log).

هنا λ = 0.66 → البيانات ليست طبيعية تمامًا، لكنها أقرب ما تكون إلى وضع يتطلب تحويل جزئي بين الخطّي واللوغاريتمي.


==================================================================================

If λ = 1→ the data is originally almost normal (no major conversion needed).

If λ = 0→ the transformation is similar to a log transformation.

Here λ = 0.66 → the data isn't completely natural, but it's as close as possible to a situation that requires a partial conversion between linear and logarithmic.

__________________________________________________________________________________

بعد هذا التحويل، توزيع الكولسترول في بياناتنا أصبح أقرب للتوزيع الطبيعي (Gaussian).

هذا مهم لأن كثيرًا من الاختبارات الإحصائية (مثل ANOVA و T-test والانحدار) تفترض أن البيانات تتبع توزيعًا طبيعيً

===========================================================================

After this conversion, the cholesterol distribution in our data is closer to the normal (Gaussian) distribution.

This is important because many statistical tests (such as ANOVA, T-test, and regression) assume that the data follow a normal distribution

In [ ]:
# 8.10 Multivariate Inspection: VIF for Multicollinearity Problem - الفحص المتعدد المتغيرات: VIF لمشكلة التعدد الخطي (Multicollinearity)
# To calculate the VIF before building a linear model. - لحساب VIF قبل بناء نموذج خطي.
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_cols = ['age', 'chol', 'trestbps', 'thalach']
X = df[num_cols].dropna()
X_const = sm.add_constant(X)

vif = pd.Series(
    [variance_inflation_factor(X_const.values, i)
    for i in range(X_const.shape[1])],
    index = X_const.columns
)
print(vif)

إذا VIF ≈ 1 → لا توجد مشكلة.

=====================================

If VIF ≈ 1→ no problem.

In [ ]:
# 8.11 Statistical Strength and Confidence Intervals (CI) Test -  اختبار القوة الإحصائية وفترات الثقة (CI)
# Confidence intervals for averages or ratios. - فترات الثقة للمتوسطات أو للنسب.
import statsmodels.stats.api as sms

# Example: CI of the mean of a sample - مثال: CI للمتوسط لعينة
cm = sms.DescrStatsW(df['age'].dropna())
print(cm.tconfint_mean()) # (low, high)


فهذا يعني أننا واثقون بنسبة 95% أن المتوسط الحقيقي لعمر المرضى يقع بين 53.36156560309382 و 55.446653574988375 سنة.

==========================================================================

This means that we are 95% confident that the true average age of patients is between 53.36156560309382 and 55.446653574988375 years.

_________________________________________________________________________

📌 ملاحظة:
افتراضيًا، tconfint_mean() يعطي فترة ثقة 95%.
إذا أردنا نسبة أخرى (مثلاً 99%)، يمكنك تعديلها:
print(cm.tconfint_mean(alpha=0.01))  # لفترة ثقة 99%

============================================================

📌 Note:
By default, tconfint_mean gives a 95% confidence interval.
If we want another percentage (e.g. 99%), you can edit it:
print(cm.tconfint_mean(alpha=0.01)) # 99% confidence interval

In [ ]:
# Bootstrap CI (General & Flexible): -  Bootstrap CI (عام ومرن):
#  Bootstrap 95% CI for Medium  -  bootstrap 95% CI للمتوسط
data = df['age'].dropna().values
n_boot = 10000
means= [np.mean(np.random.choice(data , size=len(data), replace=True)) for _ in range(n_boot)]
ci_lower, ci_upper = np.percentile(means , [2.5 , 97.5])
print(ci_lower , ci_upper)

In [ ]:
# 8.12 Analyze the relationship of variables to the goal using a simple model (logistic regression) — to interpret, not just predict -  تحليل علاقة المتغيرات بالهدف باستخدام نموذج بسيط (Logistic Regression) — للتفسير لا للتنبؤ فقط
# A logical model that gives you odds ratios and explains the effect of each variable. -   نموذج منطقي يعطيك أوزان المتغيرات (odds ratios) ويفسر تأثير كل متغير.
import statsmodels.formula.api as smf

# Suppose that Target is named Condition and as an example of using all variable  -  افترض أن target اسمه condition وكمثلة استخدام جميع المتغيرات
model = smf.logit('condition ~ age + sex + cp + trestbps + chol + fbs + restecg + thalach + exang + oldpeak + slope + ca + thal', data=df).fit()
print(model.summary())

# Probabilities and odds ratios   -    الاحتمالات والـ odds ratios
params = model.params
odds_ratios = np.exp(params)
print(odds_ratios)

### **1. جودة النموذج**

Pseudo R-squ. = 0.5035
هذا يعني أنّ النموذج يفسر تقريبًا 50% من التباين في الحالة المرضية، وهو مستوى تفسير جيد في النماذج الطبية.

LLR p-value = 3.310e-36
هذا يدلّ على أنّ النموذج ككل ذو دلالة إحصائية عالية جدًا.

### **2. معاملات المتغيرات (coef و OR)**

Intercept (المقطع): -6.2571 → ليس له معنى طبي مباشر، هو فقط نقطة بداية النموذج.

age (العمر): غير دال إحصائيًا (p=0.715)، أي أن العمر بحد ذاته ليس متغيرًا مؤثرًا قويًا بعد أخذ بقية المتغيرات بالاعتبار.

sex (الجنس): coef = 1.28 ، OR ≈ 3.60 ، p=0.009
→ الذكور (أو الفئة المشفرة 1) لديهم احتمال إصابة أعلى بحوالي 3.6 مرة مقارنة بالإناث.

cp (نوع ألم الصدر): coef = 0.59 ، OR ≈ 1.80 ، p=0.002
→ نوع ألم الصدر مرتبط بقوة بالمرض.

trestbps (ضغط الدم أثناء الراحة): coef = 0.021 ، p=0.055 (حدي borderline).
→ ارتفاع الضغط يزيد الاحتمال لكن الدلالة ضعيفة.

chol (الكولسترول): coef = 0.0091 ، OR ≈ 1.01 ، p=0.048
→ كل زيادة بمقدار 1 mg/dl في الكولسترول ترفع الاحتمال قليلًا (حوالي 1%).

fbs (سكر الدم الصائم >120mg/dl): coef = -0.96 ، OR ≈ 0.38 ، p=0.091 (غير دال).
→ الاتجاه يشير أن ارتفاع السكر قد يقلل الاحتمال بعد التحكم ببقية المتغيرات، لكن غير دال.

restecg (رسم القلب): غير دال (p=0.151).

thalach (أقصى معدل نبض): coef = -0.0198 ، OR ≈ 0.98 ، p=0.055 (حدي borderline).
→ زيادة النبض القصوى تقلل الاحتمال.

exang (ألم مع الجهد): coef = 0.85 ، OR ≈ 2.34 ، p=0.042
→ وجود ألم مع التمرين يزيد الاحتمال بأكثر من الضعف.

oldpeak (انخفاض ST): غير دال (p=0.191).

slope (ميل ST): p=0.098 (حدي borderline).

ca (عدد الأوعية الملونة): coef = 1.22 ، OR ≈ 3.39 ، p<0.001
→ من أقوى المؤشرات، زيادة عدد الأوعية غير الطبيعية يزيد الاحتمال أكثر من 3 أضعاف.

thal (اختبار thalassemia): coef = 0.75 ، OR ≈ 2.12 ، p<0.001
→ أيضًا مؤشر قوي


===================================================================================================



### **1. Model quality**



    Pseudo R-squ. = 0.5035

    This means that the model explains approximately 50% of the variance in disease status, which is a good level of explanation in medical models.



    LLR p-value = 3.310e-36

    This indicates that the model as a whole is very highly statistically significant.



### **2. Variable coefficients (coef and OR)**



    Intercept (intercept): -6.2571 → It has no direct medical meaning, it is just the starting point of the pattern.



    Age: Not statistically significant (p=0.715), meaning that age itself is not a strong influencing variable after taking other variables into account.



    sex (sex): coef = 1.28, OR ≈ 3.60, p=0.009

    → Males (or coded category 1) have a 3.6 times higher risk of infection than females.



    cp (chest pain type): coef = 0.59, OR ≈ 1.80, p = 0.002

    → The type of chest pain is strongly related to the disease.



    restbps (resting blood pressure): coef = 0.021, p = 0.055 (borderline).

    → High blood pressure increases the possibility, but the significance is weak.



    chol (cholesterol): coef = 0.0091, OR ≈ 1.01, p=0.048

    → Every 1 mg/dl increase in cholesterol increases the risk slightly (about 1%).



    FBS (fasting blood sugar >120mg/dl): coef = -0.96, OR ≈ 0.38, p=0.091 (not significant).

    → The trend suggests that high blood sugar may reduce the risk after controlling for other variables, but it is not significant.



    RestECG (Echocardiogram): Not significant (p=0.151).



    thalach (maximum pulse rate): coef = -0.0198, OR ≈ 0.98, p=0.055 (borderline).

    → Increased maximum pulse rate reduces the probability.



    exang (pain with exertion): coef = 0.85, OR ≈ 2.34, p = 0.042

    → Having pain with exercise more than doubles the likelihood.



    Oldpeak (ST depression): Not significant (p=0.191).



    slope (ST): p=0.098 (borderline).



    ca (number of colored vessels): coef = 1.22, OR ≈ 3.39, p<0.001

    → One of the strongest indicators is that an increased number of abnormal vessels increases the risk more than 3 times.



    thal (thalassemia test): coef = 0.75, OR ≈ 2.12, p<0.001

    → Also a strong indicator.



**📌 الخلاصة:**

المتغيرات الأكثر دلالة في التنبؤ بالمرض:

الجنس (sex)

نوع ألم الصدر (cp)

ألم مع الجهد (exang)

عدد الأوعية الملونة (ca)

thal
مع مساهمة إضافية (لكن أقل قوة) من الكولسترول (chol) و ضغط الدم (trestbps) و أقصى نبض (thalach).

=============================================

**📌 The Summary:**

The most significant variables in predicting disease:

    sex

    Chest pain type (CP)

    pain with exertion (exang)

    Number of colored vessels (ca)

    thal With an additional (but less powerful) contribution from cholesterol (chol), blood pressure (trestbps) and maximum pulse (thalach).

In [ ]:
# 8.13 Feature Importance / Selection  -  اختيار الخصائص (Feature Importance / Selection)
# Use simple methods: correlation ranking, mutual information or tree-based importance.   -   استخدام طرق بسيطة: correlation ranking, mutual information أو tree-based importance.
from sklearn.feature_selection import mutual_info_classif

num_cols = df.select_dtypes(include=['int64','float64']).columns.drop('condition')
X = df[num_cols].fillna(0)
y = df['condition'].astype(int)
mi = mutual_info_classif(X , y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending =False)
print(mi_series)

In [ ]:
top_features = mi_series.head(5).index
print("Top Features of Mutual Information is:", top_features.tolist())

X_selected = df[top_features]

In [ ]:
# or tree-based:  -   أو tree-based:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)
imp = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print(imp)

### **8.14 Conclusions and recommendations** - **ا**لاستنتاجات والتوصيات

#### **1- the true average age of patients is between 53.352739726027394 and 55.446653574988375 years.**


#### **2- Top Features of Mutual Information is:** 

**cp** = chest pain type (0: typical angina 1: atypical angina 2: non-anginal pain 3: asymptomatic

**ca** = number of major vessels (0-3) colored by flourosopy

**thal** = thal (0: normal; 1: fixed defect; 2: reversable defect)

**oldpeak** = ST depression induced by exercise relative to rest

**col_boxcox or chol** = serum cholestoral in mg/dl 




#### **3- Odds Ratios are simply for each parameter:**

**Age** → For each year of increase in age: the change is small and not significant (OR≈0.99 → less than 1%).

**Sex** → Males are +260% more likely to be affected (OR=3.60).

**Chest pain type (CP)** → increases the risk by about +80% (OR=1.80).

**Resting blood pressure (trestbps)** → Each 1 mmHg increase increases the risk by +2.1% (OR=1.021, borderline).

**Cholesterol (chol)** → Each 1 mg/dl increase increases the risk by +0.9% (OR=1.009).

**Fasting blood sugar (FBS)** → reduces the risk by about -62% (OR=0.38, not significant).

**Resting ECG (restecg)** → increases the risk by +30% (OR=1.30, not significant).

**Maximum heart rate (thalach)** → Each 1-beat increase reduces the probability by -1.9% (OR=0.98, borderline).

**Exercise-induced angina (exang)** → increases the risk by +134% (OR=2.34).

**ST depression (oldpeak)** → Each unit increase increases the probability by +33% (OR=1.33, not significant).

**Slope of ST segment (slope)** → increases the probability by +83% (OR=1.83, borderline).

**Number of vessels (ca)** → increases the probability by more than +239% (OR=3.39).

**Thal test (thal)** → increases the probability by +112% (OR=2.12).


=======================================================================================================================================================================================================



#### **1- متوسط ​​العمر الحقيقي للمرضى يتراوح بين 53.352739726027394 و 55.446653574988375 سنة.**


#### **2- أهم مميزات المعلومات المتبادلة هي:** 

**cp** = نوع ألم الصدر (0: ذبحة صدرية نموذجية 1: ذبحة صدرية غير نموذجية 2: ألم غير ذبحي 3: بدون أعراض

**ca** = عدد الأوعية الرئيسية (0-3) الملونة بالفلوروسكوب

**thal** = thal (0: طبيعي؛ 1: عيب ثابت؛ 2: عيب قابل للعكس)

**oldpeak** = انخفاض ST الناجم عن ممارسة الرياضة مقارنة بالراحة



#### **3- نسب الاحتمالات هي ببساطة لكل معلمة:**

**col_boxcox أو chol** = الكوليسترول في المصل بالملغ/ديسيلتر

**Age** → لكل سنة زيادة في العمر: التغير طفيف وغير دال (OR≈0.99 → أقل من 1%).

**Sex** → الذكور أكثر عرضة للإصابة بـ +260% (OR=3.60).

**Chest pain type (cp)** → يزيد الاحتمال بحوالي +80% (OR=1.80).

**Resting blood pressure (trestbps)** → كل زيادة 1 mmHg تزيد الاحتمال بـ +2.1% (OR=1.021، حدّي).

**Cholesterol (chol)** → كل زيادة 1 mg/dl تزيد الاحتمال بـ +0.9% (OR=1.009).

**Fasting blood sugar (fbs)** → يقلل الاحتمال بحوالي -62% (OR=0.38، غير دال).

**Resting ECG (restecg)** → يزيد الاحتمال بـ +30% (OR=1.30، غير دال).

**Maximum heart rate (thalach)** → كل زيادة 1 نبضة تقلل الاحتمال بـ -1.9% (OR=0.98، حدّي).

**Exercise-induced angina (exang)** → يزيد الاحتمال بـ +134% (OR=2.34).

**ST depression (oldpeak)** → كل وحدة زيادة تزيد الاحتمال بـ +33% (OR=1.33، غير دال).

**Slope of ST segment (slope)** → يزيد الاحتمال بـ +83% (OR=1.83، حدّي).

**Number of vessels (ca)** → يزيد الاحتمال بأكثر من +239% (OR=3.39).

**Thal test (thal)** → يزيد الاحتمال بـ +112% (OR=2.12).
